<a href="https://colab.research.google.com/github/thwlruss10/VDA_pipeline/blob/main/Copy_of_VILT_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.28.1 accelerate==0.21.0 peft==0.4.0

In [ ]:
!pip install transformers==4.28.1 datasets nltk scikit-learn



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from copy import deepcopy
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from datasets import load_dataset
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import re
import pandas as pd
import os
from transformers import (
    # Preprocessing / Common
    AutoTokenizer, AutoFeatureExtractor,
    # Text & Image Models & transformers (ViTModel, DeiTModel, BEiT)
    AutoModel,
    # Training / Evaluation
    TrainingArguments, Trainer,
    # Misc
    logging
)

# import nltk
# nltk.download('wordnet')

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# SET CACHE FOR HUGGINGFACE TRANSFORMERS + DATASETS
os.environ['HF_HOME'] = os.path.join(".", "cache")
# SET ONLY 1 GPU DEVICE
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

#set_caching_enabled(True)>> this line deleted as 'datasets' no longer supports explicit cahce enabling
#set_caching_enabled(True)
logging.set_verbosity_error()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [ ]:
import os
import re
import pandas as pd
from sklearn.model_selection import train_test_split

# Define base directory pointing to your Drive location
base_dir = "/content/drive/MyDrive/FinalProject/dataset"
os.makedirs(base_dir, exist_ok=True)

# Define regex to extract image ID
image_pattern = re.compile(r"( (in |on |of )?(the |this )?(image\d*) \?)")

# Read the raw Q&A file
# Extracts image ID and question at index[i] together with answer at [i+1]
# and organizes it into a pandas DataFrame, creates a list of unique answers,
# and then splits the dataset into training and testing sets, saving them as
# CSV.

qa_file = os.path.join(base_dir, "all_qa_pairs.txt")
with open(qa_file, "r", encoding="utf-8") as f: # open in read mode with in utf-8 encoding
    qa_data = [x.strip() for x in f.readlines()] # x.strip() removes white spaces

records = []
for i in range(0, len(qa_data), 2): # iterates in steps of two because quesiton is at index i and answer at i+1
    match = image_pattern.findall(qa_data[i]) # extract image ID from question string
    if match:
        img_id = match[0][3]
        question = qa_data[i].replace(match[0][0], "").strip() # remove image ID and replace question
        answer = qa_data[i + 1].strip() # answer retrieved from next element
        records.append({"question": question, "answer": answer, "image_id": img_id}) #populate dictionary



df = pd.DataFrame(records)


answer_space = []
for ans in df["answer"].to_list():
    if "," in ans:
        answer_space += ans.replace(" ", "").split(",")
    else:
        answer_space.append(ans)

# sort and remove duplicates from answer_space
answer_space = sorted(set(answer_space))

# Write the answer space to file
with open(os.path.join(base_dir, "answer_space.txt"), "w", encoding="utf-8") as f:
    f.writelines("\n".join(answer_space))

# Split dataset into training and evaluation
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save to CSV
train_df.to_csv(os.path.join(base_dir, "data_train.csv"), index=False)
test_df.to_csv(os.path.join(base_dir, "data_eval.csv"), index=False)


In [ ]:
import pandas as pd

base_dir = "/content/drive/MyDrive/FinalProject/dataset"
eval_df = pd.read_csv(os.path.join(base_dir, "data_eval.csv"))

In [ ]:
from transformers.models.vilt.modeling_vilt import ViltForQuestionAnswering
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn as nn
import torch

class ViltForSingleLabelQA(ViltForQuestionAnswering):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        pixel_values=None,
        pixel_mask=None,
        labels=None,
        return_dict=True,
        **kwargs
    ):
        # 🧼 Remove any Trainer-only args
        kwargs.pop("num_items_in_batch", None)

        # ✅ Forward to base ViLT model without passing labels (so we control loss)
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            pixel_values=pixel_values,
            pixel_mask=pixel_mask,
            return_dict=return_dict,
            **kwargs
        )

        logits = outputs.logits

        # ✅ Our own classification loss
        loss = None
        if labels is not None:
            if labels.dtype != torch.long:
                labels = labels.long()
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [ ]:
def wup_measure(pred, gt, threshold=0.9):
    pred_synsets = wn.synsets(pred)
    gt_synsets = wn.synsets(gt)

    if not pred_synsets or not gt_synsets:
        return 0.0

    max_score = max((s1.wup_similarity(s2) or 0) for s1 in pred_synsets for s2 in gt_synsets)
    return 1.0 if max_score >= threshold else max_score


In [ ]:
from transformers import ViltProcessor, ViltForQuestionAnswering
from PIL import Image
import torch

# Load model and processor
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to("cuda" if torch.cuda.is_available() else "cpu")

# Load image and define question
image_path = "/content/drive/MyDrive/FinalProject/dataset/images/image2.png"
image = Image.open(image_path).convert("RGB")
question = "What is on the left side of the sink?"

# Preprocess inputs
inputs = processor(image, question, return_tensors="pt").to(model.device)

# Inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_answer = model.config.id2label[logits.argmax(-1).item()]

print("Predicted Answer:", predicted_answer)


In [ ]:
answer_to_id = {ans: i for i, ans in enumerate(answer_space)}


missing_answers_train = train_df[~train_df["answer"].isin(answer_to_id.keys())]
missing_answers_test = test_df[~test_df["answer"].isin(answer_to_id.keys())]

print(f"Missing in train: {len(missing_answers_train)}")
print(f"Missing in test: {len(missing_answers_test)}")


In [ ]:
train_df = train_df[train_df["answer"].isin(answer_to_id.keys())].copy()
test_df = test_df[test_df["answer"].isin(answer_to_id.keys())].copy()

In [ ]:
# Map answers to class indices
# answer_to_id = {ans: i for i, ans in enumerate(answer_space)}
train_df["label"] = train_df["answer"].map(answer_to_id)
test_df["label"] = test_df["answer"].map(answer_to_id)

# Ensure labels are integers and not NaN
train_df = train_df.dropna(subset=["label"]).copy()
test_df = test_df.dropna(subset=["label"]).copy()

train_df["label"] = train_df["label"].astype(int)
test_df["label"] = test_df["label"].astype(int)



In [ ]:

from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
})


In [ ]:

from transformers import ViltProcessor
from PIL import Image

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")
image_dir = os.path.join(base_dir, "images")

def preprocess(example):
    image_path = os.path.join(image_dir, f"{example['image_id']}.png")
    image = Image.open(image_path).convert("RGB")

    # Use return_tensors="np" to avoid PyTorch tensor overload
    inputs = processor(
        text=example["question"],
        images=image,
        return_tensors="np",
        padding="max_length",
        truncation=True,
    )

    # Flatten tensors to remove batch dim
    inputs = {k: v.squeeze(0) if hasattr(v, "shape") and v.shape[0] == 1 else v for k, v in inputs.items()}
    inputs["labels"] = int(example["labels"])
    return inputs


In [ ]:
dataset = dataset.rename_column("label", "labels")



In [ ]:
# Apply pre-processing
#dataset = dataset.map(preprocess)
dataset = dataset.map(preprocess, remove_columns=dataset["train"].column_names)

print(dataset["train"][0]["labels"], type(dataset["train"][0]["labels"]))

In [ ]:
# Try 500 examples
medium_dataset = DatasetDict({
    "train": dataset["train"].select(range(500)),
    "test": dataset["test"].select(range(100)),
})


In [ ]:
# Load the model

from transformers import ViltForQuestionAnswering
import os  # <-- missing import

# Define the base directory (your dataset folder in Google Drive)
base_dir = "/content/drive/MyDrive/FinalProject/dataset"

# Load the list of all possible answers
with open(os.path.join(base_dir, "answer_space.txt"), "r", encoding="utf-8") as f:
    answer_space = f.read().splitlines()

In [ ]:
from transformers import TrainingArguments

training_argsX = TrainingArguments(
    output_dir="./test_output",
    evaluation_strategy="steps"
)

print("✅ TrainingArguments working.")

In [ ]:
model = ViltForSingleLabelQA.from_pretrained(
    "dandelin/vilt-b32-mlm",
    num_labels=len(answer_space)
)
model.config.problem_type = "single_label_classification"

In [ ]:
from sklearn.metrics import accuracy_score

from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    preds = np.argmax(logits, axis=-1)

    # Decode IDs into answer strings
    pred_answers = [answer_space[p] for p in preds]
    true_answers = [answer_space[l] for l in labels]

    # WUPS across all pairs
    wups = [
        wup_measure(pred, true)
        for pred, true in zip(pred_answers, true_answers)
    ]

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="macro"),
        "wups": np.mean(wups)
    }

In [ ]:
from transformers import TrainingArguments

training_argsX = TrainingArguments(
    output_dir="./test_output",
    evaluation_strategy="steps"
)

print("✅ TrainingArguments working.")

In [ ]:
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModel
from transformers import logging
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./vilt_daquar_output",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
    #label_smoothing_factor=0.1,  # ✅ Try 0.1 to start
    remove_unused_columns=False  # ✅ ADD THIS LINE
)


In [ ]:
from transformers import default_data_collator

def data_collator(features):
    batch = default_data_collator(features)
    batch["labels"] = batch["labels"].long()  # 💥 force long tensor
    return batch

In [ ]:
from transformers import TrainingArguments

training_argsX = TrainingArguments(
    output_dir="./test_output",
    evaluation_strategy="steps"
)

print("✅ TrainingArguments working.")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=medium_dataset["train"],
    eval_dataset=medium_dataset["test"],
    tokenizer=processor,
    data_collator=data_collator,  # <-- custom one
    compute_metrics=compute_metrics,
)

In [ ]:
model.config.problem_type = "single_label_classification"


In [ ]:
from nltk.corpus import wordnet as wn
trainer.train()

In [ ]:
trainer.save_model("/content/drive/MyDrive/FinalProject/vilt_checkpoint")
processor.save_pretrained("/content/drive/MyDrive/FinalProject/vilt_checkpoint")

In [ ]:
dataset.save_to_disk("/content/drive/MyDrive/FinalProject/vilt_preprocessed")

In [ ]:
metrics = trainer.evaluate()
with open("/content/drive/MyDrive/FinalProject/metrics.json", "w") as f:
    import json
    json.dump(metrics, f, indent=2)

In [ ]:
def print_parameter_count(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"🧠 Total parameters:     {total:,}")
    print(f"🎯 Trainable parameters: {trainable:,}")
    print(f"🪶 Frozen parameters:    {total - trainable:,}")

print_parameter_count(model)